In [2]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.llms import Ollama 
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [4]:
loader = PyPDFLoader("mg_pdf.pdf")
pdf_doc = loader.load()


In [12]:
txt_split =  RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
doc_split = txt_split.split_documents(pdf_doc)


In [15]:
db1 = FAISS.from_documents(doc_split[:30],OllamaEmbeddings(model="llama3.1"))

In [19]:
query =  "Give me the traduction of Accréditation from francais to malgache"
res = db1.similarity_search(query)
res[0].page_content

"Accord ministériel reçu  Fahazahoana ny fankatoavan'ny \nMinisitra  \nAccord multilatéral   Fifanarahana maro lafy  \nAccord préalable  Fanekena  mialoha, fifanekena \nmialoha  \nAccord subsidiaire  Fifanekena lefitra  \nAccord -cadre   Fifanarahana lasitra  \nAccréditation  Fanomezam -pahefana  \nAccréditer  Manome fahefana, manendry \nmasoivoho, manendry \n(ambasadaoro)  \nAccroissement  Fitomboana fanitarana, fihitarana, \nfampitomboana  \nAccroissement de capital  Fitomboan'ny renivola  \nAccroître  Mampit ombo, manitatra  \nAccroupie  Mitalapetraka, mitorovoka  \nAccumulateur (véhicule)  Tobin'aratra(fiarakodia), \nmpanangona araka aratra"

In [34]:
llm = Ollama(model='llama3.1')
prompt = ChatPromptTemplate.from_template(""" Find The Best Traduction from french to malagasy
or Malagasy to French based on the user word from the context only.
<context>
{context}
Word : {input} """)

In [35]:
doc_chain  = create_stuff_documents_chain(llm,prompt)
retriever = db1.as_retriever()
retriever_chain = create_retrieval_chain(retriever,doc_chain)

In [36]:
res = retriever_chain.invoke({"input" : "Bonjour"})


In [37]:
print(res['answer'])

Le traduction de "Bonjour" du français vers le malagasy est :

* Salama

Notez que j'ai utilisé l'outil de traduction automatique et que la traduction peut varier en fonction du contexte. Dans ce cas, je me suis basé sur le contexte du livre qui décrit un dictionnaire de traduction français-malgache, donc j'ai choisi une traduction courante et familière dans les deux langues.

Si vous voulez savoir la traduction du mot "Bonjour" du malagasy vers le français, je réponds :

* Bonjour

ou

* Salama (dans un contexte familier)

Notez que le mot "Salama" est souvent utilisé pour saluer dans les deux langues, mais il peut avoir des connotations légèrement différentes en fonction du contexte.


In [38]:
print(res['answer'].replace("\n\n", "\n").replace("**", "").replace("* ", "- "))


Le traduction de "Bonjour" du français vers le malagasy est :
- Salama
Notez que j'ai utilisé l'outil de traduction automatique et que la traduction peut varier en fonction du contexte. Dans ce cas, je me suis basé sur le contexte du livre qui décrit un dictionnaire de traduction français-malgache, donc j'ai choisi une traduction courante et familière dans les deux langues.
Si vous voulez savoir la traduction du mot "Bonjour" du malagasy vers le français, je réponds :
- Bonjour
ou
- Salama (dans un contexte familier)
Notez que le mot "Salama" est souvent utilisé pour saluer dans les deux langues, mais il peut avoir des connotations légèrement différentes en fonction du contexte.
